In [1]:
import cv2
import imutils
import os
import numpy as np
import psutil
from helper_functions import extract_patches

In [2]:
hwsize = 32

# Arrays of rotations to be used for patch creation. 
pos_rots = [0, 45, 90]
neg_rots = [0]

# Lists for the datapoints with no centers and no blue ratio data respectively
no_centers = []
no_br = []

# Dictionaries where the labels for each patch will be stored. Needed for training and testing
dict_labels_pos = {}
dict_labels_neg = {}

# Original dataset location
orig_dataset_dir = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/mitosis/'
patches_dataset_dir = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/'
os.chdir(orig_dataset_dir)
root =os.path.splitext(orig_dataset_dir)[0]
print(root)

/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/mitosis/


## !! Caution: Takes a long time to execute

In [3]:
for file in os.listdir(orig_dataset_dir):
#     print(file)
    if file.endswith(".tif"):
        
#         print(file)
        print("ram usage: " + str(psutil.virtual_memory().percent))
        basename = os.path.splitext(file)[0]
        filename = os.path.join(root + file)
        
                
        image = cv2.imread(file, cv2.IMREAD_UNCHANGED)
        
        if type(image) == type(None):
        
            print("Error: file " + file + " not read")
        
        else:
            
            # Resize the image to 20x magnification (initially it was 2000x2000)
            # The reason behind this is to ensure that the entire mitotic center is contained in a patch
            image = cv2.resize(src=image, dsize=(1000,1000))
            
            # Form the name of the Ground Truth file
            # Will later check if it actually exists
            gt_maskname = os.path.join(root + basename + "_pc.png")

            # Read the Ground Truth mask
            gt_mask = cv2.imread(gt_maskname, cv2.IMREAD_UNCHANGED)

            # Check if the image has a ground truth associated
            if type(gt_mask) is type(None):

                # Add to a list of missing ground truths
                no_centers.append(basename)
                print("no GT " + os.path.split(gt_maskname)[1])

            else:

                # Resize the image to 20x magnification
                # The reason behind this is to ensure that the entire mitotic center is contained in a patch
                gt_mask = cv2.resize(src=gt_mask, dsize=(1000,1000))
                
                # Extract the coordinates of the centers 
                gt_result = np.where(gt_mask == 255)
#                 print("number of GT pos pix in " + gt_maskname + "= " + str(len(gt_result[0])))
                
                # Extract the patches surrounding the positive pixels
                extract_patches(image, gt_result, 'class1', pos_rots, hwsize, dict_labels_pos, patches_dataset_dir, basename)



            # Form the name of the Blue Ratio Segmentation mask file
            br_maskname = os.path.join(root + basename + "_cmask.png")

            # Read the Blue Ratio Segmentation mask
            br_mask = cv2.imread(br_maskname, cv2.IMREAD_UNCHANGED)

            # Check if the image has a BR mask associated
            if type(br_mask) == type(None):

                # Add to a list of missing BR masks
                no_br.append(basename)
                print("no BR " + str(br_maskname))

            else:

                # Resize the image to 20x magnification
                # The reason behind this is to ensure that the entire mitotic center is contained in a patch
                br_mask = cv2.resize(src=br_mask, dsize=(1000,1000))        
        
                # Extract the coordinates of the centers 
                br_result = np.where(br_mask == 255)
#                 print("Size of BR mask before removing pos pix: " + str(len(br_result[0])) + ", " + str(len(br_result[1])))

                # Check if the Ground Truth mask exists (ie. if there are any positive centers in the current image)
                if ~(type(gt_mask) is type(None)):

                    # Remove the positive centers from the BR mask
                    for id1, id2 in zip(gt_result[0], gt_result[1]):
                        br_mask[id1][id2] = 0

                    # Extract the coordinates of the centers 
                    br_result = np.where(br_mask == 255)
#                     print("Size of BR mask after removing pos pix: " + str(len(br_result[0])) + ", " + str(len(br_result[1])))

                # Extract the patches whether we had positive centers or not
                # These ones will be used as negative samples
                extract_patches(image, br_result, 'class0', neg_rots, hwsize, dict_labels_neg, patches_dataset_dir, basename)

         
    # break "file" loop
#     break
    
print("\nThe following "+ str(len(no_centers)) +" datapoints contain no Ground Truth info: ")
print(no_centers)

print("\nThe following "+ str(len(no_br)) +" datapoints contain no Blue Ration Segmentation info: ")
print(no_br)
            
# Free up the space
no_centers = None
no_br = None

ram usage: 76.3
class1 number of pix after edge filtering= 134
 final number of pix to be extracted:  134
class0 number of pix after edge filtering= 94208
 final number of pix to be extracted:  188
ram usage: 76.4
class1 number of pix after edge filtering= 67
 final number of pix to be extracted:  67
class0 number of pix after edge filtering= 122992
 final number of pix to be extracted:  246
ram usage: 76.4
class1 number of pix after edge filtering= 134
 final number of pix to be extracted:  134
class0 number of pix after edge filtering= 98211
 final number of pix to be extracted:  196
ram usage: 76.4
class1 number of pix after edge filtering= 0
 final number of pix to be extracted:  0
class0 number of pix after edge filtering= 175888
 final number of pix to be extracted:  352
ram usage: 76.4
class1 number of pix after edge filtering= 67
 final number of pix to be extracted:  67
class0 number of pix after edge filtering= 112199
 final number of pix to be extracted:  224
ram usage: 76.2

ram usage: 76.5
class1 number of pix after edge filtering= 67
 final number of pix to be extracted:  67
class0 number of pix after edge filtering= 197936
 final number of pix to be extracted:  396
ram usage: 76.5
class1 number of pix after edge filtering= 134
 final number of pix to be extracted:  134
class0 number of pix after edge filtering= 145164
 final number of pix to be extracted:  290
ram usage: 76.5
class1 number of pix after edge filtering= 134
 final number of pix to be extracted:  134
class0 number of pix after edge filtering= 118447
 final number of pix to be extracted:  237
ram usage: 76.5
class1 number of pix after edge filtering= 67
 final number of pix to be extracted:  67
class0 number of pix after edge filtering= 65527
 final number of pix to be extracted:  131
ram usage: 76.5
class1 number of pix after edge filtering= 67
 final number of pix to be extracted:  67
class0 number of pix after edge filtering= 127513
 final number of pix to be extracted:  255
ram usage: 7

In [ ]:
# image = cv2.imread('/home/advo/PycharmProjects/Udacity_CapstoneProject/Dataset/mitosis/01_01.tif', cv2.IMREAD_UNCHANGED)
mask = cv2.imread('/home/advo/PycharmProjects/Udacity_CapstoneProject/Dataset/mitosis/01_01_cmask.png', cv2.IMREAD_UNCHANGED)
# print(mask.shape)

In [ ]:
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
image = cv2.resize(src=image, dsize=(1000,1000))
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# result = np.where(mask == 255)

In [ ]:
# # Get the row and column coordinates of the positive pixels
# rows = result[0]
# cols = result[1]

# # Get the size of the image (width, height)
# imsize_w = image.shape[0]
# imsize_h = image.shape[1]

In [ ]:
# # Get the size of the positive pixels array
# npos = len(rows)

In [ ]:
# # Establish the condition based on which the positive pixels will be kept
# # ie. remove the pixels that are located at any of the borders of the image
# # because they cannot form valid patches
# # Note. 2*hwsize is used because the initial patches will be double the size, since we want to be able to rotate them
# condition_row = (rows + 2*hwsize <= imsize_h) & (rows - 2*hwsize >= 0)
# condition_col = (cols + 2*hwsize <= imsize_w) & (cols - 2*hwsize >= 0)

In [ ]:
# # Remove the border pixels; keep the original rows, cols
# rows_filt = rows[condition_row]
# cols_filt = cols[condition_col]

In [ ]:
# idx = 1
# # Extract the 64x64 patch 
# patch = image[rows_filt[idx]-hwsize:rows_filt[idx]+hwsize, cols_filt[idx]-hwsize:cols_filt[idx]+hwsize]

In [ ]:
cv2.imshow("image", patch)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
patch.shape

In [ ]:
# rot = [0, 45, 90, 135, 180]
# # rpatch = cv2.rotate(patch, cv2.ROTATE_90_CLOCKWISE)
# rpatch = imutils.rotate(patch, angle = rot[3])
# rpatch_resized = rpatch[16:48, 16:48, :]  

In [ ]:
cv2.imshow("image2", rpatch_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
rpatch_resized.shape

In [ ]:
arr1 = [[0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0]]

In [ ]:
r = [1, 2, 0]
c = [0, 1, 0]

In [ ]:
for id1,id2 in zip(r,c):
    arr1[id1][id2] = 0 

In [ ]:
arr1